In [327]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [328]:
from STOCK_PREDICT.data import Data

# other libraries
from tensorflow import keras
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding,Conv1D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import sequence
import os
import nltk
from keras.utils import to_categorical

In [329]:
data = Data()
data = data.clean_df()
data.keys()

Index(['Date', 'Label', 'Top1', 'Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7',
       'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15',
       'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23',
       'Top24', 'Top25', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close',
       'change', 'target', 'combined', 'cleaned', 'polarity', 'subjectivity',
       'compound', 'negative', 'neutrale', 'positive'],
      dtype='object')

In [330]:
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,change,target,combined,cleaned,polarity,subjectivity,compound,negative,neutrale,positive
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,0.026030,1,"b""Georgia 'downs two Russian warplanes' as cou...",b georgia down two russian warplane country mo...,-0.047536,0.254960,-0.9971,0.286,0.647,0.067
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,0.004436,1,b'Why wont America and Nato help us? If they w...,b wont america nato help u wont help u help ir...,0.113565,0.346443,-0.9895,0.283,0.614,0.103
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,-0.012637,0,b'Remember that adorable 9-year-old who sang a...,b remember adorable year old sang opening cere...,-0.026211,0.506694,-0.9942,0.258,0.651,0.091
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,-0.008660,0,b' U.S. refuses Israel weapons to attack Iran:...,b u refuse israel weapon attack iran report pr...,0.014531,0.335034,-0.9712,0.198,0.698,0.103
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree to missle defense deal. ...,"b'Will the Russians conquer Tblisi? Bet on it,...",...,0.006863,1,b'All the experts admit that we should legalis...,b expert admit legalise drug war south osetia ...,0.037480,0.344004,-0.9855,0.248,0.621,0.130


In [59]:
df = data['cleaned']

## Word2Vec Deep

In [147]:
tokens = nltk.word_tokenize(str(df.tolist()))

In [148]:
# Solution
dictionnary = {}
i = 0
frequency = {}
for x in tokens:
    if x not in dictionnary:
        dictionnary.update({x:i})
        i = i + 1
    
    if x in frequency:
        frequency[x] += 1
    else:
        frequency[x] = 1

In [229]:
frequency

{'[': 1,
 "'Georgia": 1,
 'two': 793,
 'Russian': 767,
 'warplane': 44,
 'country': 1526,
 'move': 284,
 'brink': 46,
 'war': 996,
 'Musharraf': 6,
 'impeached': 2,
 'Today': 58,
 'Columns': 1,
 'troop': 448,
 'roll': 22,
 'South': 631,
 'Ossetia': 30,
 'footage': 84,
 'fighting': 161,
 'YouTube': 55,
 'tank': 115,
 'moving': 55,
 'towards': 94,
 'capital': 201,
 'reportedly': 120,
 'completely': 51,
 'destroyed': 103,
 'Georgian': 26,
 'artillery': 39,
 'child': 973,
 'raped': 134,
 'official': 828,
 'say': 2763,
 'sick': 36,
 'three': 415,
 'year': 1906,
 'old': 309,
 'nothing': 85,
 'entered': 49,
 'whilst': 6,
 'Georgia': 91,
 'shoot': 98,
 'jet': 165,
 'invades': 6,
 'Russia': 856,
 'warned': 162,
 'would': 776,
 'intervene': 21,
 'SO': 2,
 'side': 105,
 'combatent': 1,
 'trial': 234,
 'sham': 7,
 'Salim': 2,
 'Haman': 1,
 'sentenced': 239,
 'kept': 67,
 'longer': 176,
 'anyway': 10,
 'feel': 82,
 'like': 488,
 'retreat': 19,
 'Osettain': 1,
 'presumably': 3,
 'leaving': 106,
 'se

In [230]:
freq = {}
for key, values in frequency.items(): 
    if value > 50:
        freq.update({item:values})

In [231]:
# Keep only words above a specific threshold
from collections import Counter
main_dict = Counter(frequency)  
trsh = 300

main_dict = Counter(dict(filter(lambda x: x[1] >= trsh, main_dict.items())))

In [232]:
main_dict

Counter({"'": 2099,
         ',': 1988,
         'A': 1181,
         'Afghan': 314,
         'Afghanistan': 399,
         'Africa': 340,
         'American': 448,
         'Arabia': 361,
         'Australian': 317,
         'Bank': 395,
         'Britain': 310,
         'British': 569,
         'Canada': 401,
         'China': 1083,
         'Chinese': 644,
         'EU': 464,
         'Egypt': 340,
         'Europe': 474,
         'European': 505,
         'For': 343,
         'France': 346,
         'French': 414,
         'Gaza': 705,
         'German': 443,
         'Germany': 440,
         'I': 875,
         'In': 771,
         'India': 499,
         'Internet': 324,
         'Iran': 727,
         'Iraq': 487,
         'Is': 460,
         'Islamic': 467,
         'Israel': 1281,
         'Israeli': 873,
         'It': 624,
         'Japan': 376,
         'Korea': 1077,
         'Korean': 386,
         'London': 306,
         'Mexico': 324,
         'Minister': 556,
         'Musli

In [233]:
# Tokenize dataframe
def tokenization(X):
    tokens = nltk.word_tokenize(X)
    return tokens

new = df.apply(tokenization)

In [234]:
def clean(test):
    for i in test:
        if i not in main_dict:
            test.remove(i)
    return test

new = new.apply(clean)

In [159]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [174]:
def concat():
    for i in liste:
        x = concat(i)
    return x

In [179]:
new = pd.DataFrame(new)

# Create categories based on words

In [279]:
def Russia(x):
    if "US" and "crisis" in x:
        x =  1
    else:
        x = 0
    return x

In [294]:
data[data['change'] < - 0.05]['cleaned'][62]

'Smokers banned fostering Suddenly It May Be Cool To Be American Again Pirate pride Sweden Pirate Bay hit million peer help third world country improve long term kind tight as Try asked take people fingerprint ID Card scheme freedom threat say US DEA killed Bolivian ID Smith fingerprint borrowed need pop supermarket Mideast peace pact unlikely end year I hate cynical really expected happen Osama bin Laden loses asylum bid Spain Canada love USA today We NEVER President Hugo Chavez government say taking one world largest gold deposit A Nepali teacher finally turned dream reality connecting remote mountain community internet Oxford grad becomes Bhutan new king world youngest monarch Warns Missile Deployment raided selling herbal ecstacy internet usage one lowest Europe Does make backwards bomber hit Pakistan tribal Hitmen Finish Victim During Surgery passport fee jump third pay High Commissioner Human Rights told BBC taken aback terrible condition Gaza recent magazine banned Islam Hunting

In [280]:
check = df.apply(Russia)
check

0       1
1       0
2       0
3       0
4       1
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      1
       ..
1959    0
1960    1
1961    0
1962    0
1963    0
1964    1
1965    1
1966    0
1967    0
1968    0
1969    0
1970    0
1971    0
1972    0
1973    0
1974    0
1975    1
1976    0
1977    0
1978    0
1979    1
1980    0
1981    0
1982    0
1983    0
1984    0
1985    0
1986    0
1987    0
1988    1
Name: cleaned, Length: 1989, dtype: int64

## Correlation

## Topic modelling

In [177]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(new)

lda_model = LatentDirichletAllocation(n_components=10)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [178]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('cleaned', 0.10004566951507017)]
Topic 1:
[('cleaned', 0.1000456225702397)]
Topic 2:
[('cleaned', 0.10004562205526166)]
Topic 3:
[('cleaned', 0.1000456276132675)]
Topic 4:
[('cleaned', 0.10004562268172741)]
Topic 5:
[('cleaned', 0.10004564550219014)]
Topic 6:
[('cleaned', 0.10004562545316573)]
Topic 7:
[('cleaned', 1.0995892955858433)]
Topic 8:
[('cleaned', 0.10004564687087564)]
Topic 9:
[('cleaned', 0.10004562215235857)]


## CHECK IT

In [438]:
train = data[data['Date'] < '20150101']
test = data[data['Date'] < '20141213']
train.shape, test.shape

((1611, 43), (1989, 43))

In [439]:
X_train = train.iloc[:,-7:]
X_test = test.iloc[:,-7:]

In [440]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [441]:
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))

In [442]:
X_train = vectorizer.fit_transform(X_train['cleaned'])


In [443]:
X_train = X_train.toarray()

In [444]:
X_test = vectorizer.transform(X_test['cleaned'])
X_test = X_test.toarray()

In [445]:
X_train.shape , X_test.shape

((1611, 26311), (1989, 26311))

In [446]:
other_train = train.iloc[:,-6:]
other_test = test.iloc[:,-6:]
other_train.shape, X_train.shape

((1611, 6), (1611, 26311))

In [447]:
other_train = np.array(other_train)
other_test = np.array(other_test)

In [448]:
other_train.shape, X_train.shape

((1611, 6), (1611, 26311))

In [449]:
full_train = np.concatenate((other_train,X_train),axis=1)
full_test = np.concatenate((other_test,X_test),axis=1)

In [450]:
y_train = train['Label']
y_test = test['Label']

In [451]:
full.shape, y.shape

((1989, 28766), (1989,))

In [452]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(full, y, test_size=0.33, random_state=42)

In [453]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200,criterion='entropy')
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [454]:
clf.score(X_test, y_test)

0.4992389649923896

In [326]:
pd.DataFrame(full)

,0,1,2,3,4,5,6,7,8,9,...,28893,28894,28895,28896,28897,28898,28899,28900,28901,28902
0,-0.045117,0.263801,-0.9967,0.297,0.629,0.074,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.084858,0.355152,-0.9773,0.259,0.644,0.097,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.039540,0.526413,-0.9932,0.276,0.622,0.102,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001144,0.382353,-0.9678,0.204,0.692,0.104,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.011420,0.350108,-0.9788,0.253,0.615,0.132,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.041201,0.448654,-0.9879,0.295,0.580,0.126,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.093147,0.473140,-0.9944,0.355,0.576,0.069,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,-0.050379,0.223485,-0.9930,0.289,0.626,0.085,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.039918,0.239838,-0.9926,0.289,0.635,0.076,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.052381,0.269048,-0.9936,0.262,0.639,0.099,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
